In [2]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize

# Gaussian Process와 관련 함수들
class GaussianProcess:
    def __init__(self, kernel, noise=1e-8):
        self.kernel = kernel
        self.noise = noise
        
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        self.K = self.kernel(self.X_train, self.X_train) + self.noise * np.eye(len(self.X_train))
        self.L = np.linalg.cholesky(self.K)
        self.alpha = np.linalg.solve(self.L.T, np.linalg.solve(self.L, self.y_train))
        
    def predict(self, X_test):
        K_s = self.kernel(self.X_train, X_test)
        K_ss = self.kernel(X_test, X_test)
        
        Lk = np.linalg.solve(self.L, K_s)
        mu = K_s.T.dot(self.alpha)
        s2 = np.diag(K_ss) - np.sum(Lk**2, axis=0)
        return mu, s2

def rbf_kernel(X1, X2, l=1.0, sigma_f=1.0):
    sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    return sigma_f**2 * np.exp(-0.5 / l**2 * sqdist)

def expected_improvement(X, X_sample, gp, y_sample, xi=0.01):
    mu, sigma = gp.predict(X)
    mu_sample = gp.predict(X_sample)[0]
    
    sigma = sigma.reshape(-1, 1)
    mu_sample_opt = np.max(mu_sample)
    
    with np.errstate(divide='warn'):
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
        
    return ei

def propose_location(acquisition, X_sample, Y_sample, gp, bounds, n_restarts=25):
    dim = X_sample.shape[1]
    min_val = 1
    min_x = None
    
    def min_obj(X):
        return -acquisition(X.reshape(1, -1), X_sample, gp, Y_sample).ravel()
    
    for x0 in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_restarts, dim)):
        res = minimize(min_obj, x0=x0, bounds=bounds, method='L-BFGS-B')
        if res.fun < min_val:
            min_val = res.fun
            min_x = res.x
            
    return min_x.reshape(1, -1)

# 메인 최적화 루프
def bayesian_optimization(n_iters, objective_function, bounds):
    dim = bounds.shape[0]
    X_sample = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(10, dim))
    Y_sample = objective_function(X_sample)
    
    gp = GaussianProcess(kernel=rbf_kernel)
    
    for i in range(n_iters):
        gp.fit(X_sample, Y_sample)
        
        # 다음 샘플링 위치 제안
        next_sample = propose_location(expected_improvement, X_sample, Y_sample, gp, bounds)
        
        # 새 샘플에 대한 목적 함수 계산
        next_sample_value = objective_function(next_sample)
        
        # 샘플 데이터에 새 포인트 추가
        X_sample = np.vstack((X_sample, next_sample))
        Y_sample = np.concatenate([Y_sample, next_sample_value])
        
        print(f"Iteration {i+1}/{n_iters}, Best error: {Y_sample.min()}")
    
    return X_sample[Y_sample.argmin()]

# 예시 목적 함수 (실제 모델을 여기에 연결해야 함)
def objective_function(X):
    return np.sum(X**2, axis=1).reshape(-1, 1)  # 간단한 예시, 실제로는 모델의 에러를 반환해야 함

# 최적화 실행
bounds = np.array([[-10, 10]] * 507)  # 각 파라미터의 범위를 -10에서 10으로 설정
best_params = bayesian_optimization(100, objective_function, bounds)

print("Best parameters found:", best_params)
print("Best error:", objective_function(best_params.reshape(1, -1))[0])

Iteration 1/100, Best error: 16123.481580959848
Iteration 2/100, Best error: 16123.481580959848
Iteration 3/100, Best error: 16063.306271087891
Iteration 4/100, Best error: 16063.306271087891
Iteration 5/100, Best error: 16063.306271087891
Iteration 6/100, Best error: 16063.306271087891
Iteration 7/100, Best error: 16063.306271087891
Iteration 8/100, Best error: 16063.306271087891
Iteration 9/100, Best error: 16063.306271087891
Iteration 10/100, Best error: 16063.306271087891
Iteration 11/100, Best error: 16063.306271087891
Iteration 12/100, Best error: 16063.306271087891
Iteration 13/100, Best error: 16063.306271087891
Iteration 14/100, Best error: 16063.306271087891
Iteration 15/100, Best error: 16063.306271087891
Iteration 16/100, Best error: 16063.306271087891
Iteration 17/100, Best error: 16063.306271087891
Iteration 18/100, Best error: 15767.777240060635
Iteration 19/100, Best error: 15767.777240060635
Iteration 20/100, Best error: 15767.777240060635
Iteration 21/100, Best error:

KeyboardInterrupt: 